This is an example usage of VI with a PyTorch model.

In [1]:
!pip freeze

absl-py==1.3.0
anyio @ file:///tmp/build/80754af9/anyio_1644463572971/work/dist
aqtp==0.0.10
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///tmp/build/80754af9/argon2-cffi-bindings_1644569679365/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
astunparse==1.6.3
attrs @ file:///croot/attrs_1668696182826/work
Babel @ file:///tmp/build/80754af9/babel_1620871417480/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///opt/conda/conda-bld/beautifulsoup4_1650462163268/work
bleach @ file:///opt/conda/conda-bld/bleach_1641577558959/work
brotlipy==0.7.0
cached-property==1.5.2
cachetools==5.2.0
certifi @ file:///croot/certifi_1671487769961/work/certifi
cffi @ file:///tmp/abs_98z5h56wf8/croots/recipe/cffi_1659598650955/work
charset-normalizer==2.1.1
chex==0.1.5
cloudpickle==2.2.0
clu==0.0.7
commonmark==0.9.1
contextlib2==21.6.0
contourpy==1.0.6
cryptography @ file:///croot/cr

First clone the repo of the model and checkout our customized branch to see
the changes required by the VI.

In [ ]:
!git clone https://github.com/stamatiad/dino.git
%cd dino
!git checkout stamatiad

As you can see, we have created a wrapper function (save_attn_weights) that
wraps the forward method of Attention. Now each time the forward method is
called and we have VI recording enabled (with vi.enable_vi() context manager)
, we will save the TB summaries in the directory ./vi_logs.

In [ ]:
!git diff main..stamatiad -- vision_transformer.py

Make sure you have installed the required python packages.

In [ ]:
!pip install -r requirements_colab.txt

If you happened to have issues with packages' versions and you restarted your
runtime, make sure you are in the right folder:

In [ ]:
%cd dino

Now lets run our example

In [ ]:
# Work on the original DINO with PyTorch:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms as pth_transforms
from vision_transformer import VisionTransformer
from PIL import Image
import requests
from io import BytesIO
from pathlib import Path


# VI imports:
import tensorflow as tf
import json
from vit_inspect import vit_inspector as vi
from vit_inspect.summary_v2 import vi_summary

Load the pre-trained model and transfer its parameters to a new instance of
our modified model, that VI listens to.

In [ ]:
# Load the pre-trained model:
model_cached = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
# Create a version of the model that holds our attention VI modifications:
# Match model params, before load:
num_features = model_cached.embed_dim
model = VisionTransformer(embed_dim=num_features)
model.load_state_dict(model_cached.state_dict(), strict=False)

# Enable evaluation mode:
device = torch.device("cpu")
for p in model.parameters():
    p.requires_grad = False
model.eval()
model.to(device)

Initialize the VI and inform it about our model parameters:

In [ ]:
# Get some model params, required for VI:
vi.params["num_layers"] = len(model.blocks)
vi.params["num_heads"] = model.blocks[0].attn.num_heads
# The number of tokens when the attention dot product happens.
# Here tokens are the patches. Any other feature (e.g. class) is removed.
patch_size = model.patch_embed.patch_size
crop_size = 480
img_size_in_patches = crop_size // patch_size
vi.params["len_in_patches"] = img_size_in_patches
# Total patches in the image:
vi.params["num_tokens"] = img_size_in_patches ** 2


Load a sample image to calculate attention maps uppon.

In [ ]:
# Load sample images:
response = requests.get("https://dl.fbaipublicfiles.com/dino/img.png")
img = Image.open(BytesIO(response.content))
img = img.convert('RGB')

# Perform the original transformations that the authors did.
transform = pth_transforms.Compose([
    pth_transforms.Resize(img.size),
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
img = transform(img)
# make the image divisible by the patch size
w, h = img.shape[1] - img.shape[1] % patch_size, \
       img.shape[2] - img.shape[2] % patch_size
img = img[:, :w, :h].unsqueeze(0)

w_featmap = img.shape[-2] // patch_size
h_featmap = img.shape[-1] // patch_size


Save a copy of the input image for the VI to display it as preview, making it
 easier to visualize the attention maps.

In [ ]:
# Save the input image into the summary:
flat_arr_rgb = tf.convert_to_tensor(
    # Make sure image's channels is the last dim:
    np.moveaxis(np.asarray(img), 1, -1)
)
with vi.writer.as_default():
    step = 0
    batch_id = 0
    vi.params["step"] = 0
    vi.params["batch_id"] = batch_id
    vi_summary(
        f"b{batch_id}",
        flat_arr_rgb,
        step=step,
        description=json.dumps(vi.params)
    )
    vi.writer.flush()


Finally, perform inference with VI enabled:

In [ ]:
# Use the VI context manager to get attention maps of each layer and head:
with vi.enable_vi():
    attentions = model.get_last_selfattention(img.to(device))


In [ ]:
%load_ext tensorboard
%tensorboard --logdir vi_logs